In [2]:
import jieba
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def load_stopwords(stopwords_path):
    with open(stopwords_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f]
    
def preprocess_data(corpus_path, stopwords):
    corpus = []
    with open(corpus_path, 'r', encoding='utf-8') as f:
        for line in f:
            corpus.append(' '.join([word for word in jieba.lcut(line.strip()) if word not in stopwords]))
    return corpus

n = 1000
stopwords_path = "../data/stop_words.txt"
documents_path = "../data/documents_first_" + str(n) + ".txt"
stopwords = load_stopwords(stopwords_path)
documents = preprocess_data(documents_path, stopwords)

Building prefix dict from the default dictionary ...
2020-06-19 10:09:35,883 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/m3/4yh806w92fdgcn0bk16ql7nw0000gn/T/jieba.cache
2020-06-19 10:09:35,886 : DEBUG : Loading model from cache /var/folders/m3/4yh806w92fdgcn0bk16ql7nw0000gn/T/jieba.cache
Loading model cost 0.630 seconds.
2020-06-19 10:09:36,514 : DEBUG : Loading model cost 0.630 seconds.
Prefix dict has been built successfully.
2020-06-19 10:09:36,515 : DEBUG : Prefix dict has been built successfully.


In [4]:
from gensim import corpora, models, similarities
from pprint import pprint
from collections import defaultdict

In [6]:
texts = [[word for word in document.split()] for document in documents]

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 2] for text in texts]

dictionary = corpora.Dictionary(texts)
dictionary.save('../data/first_' + str(n) + '_doc.dict')
print(dictionary)

2020-06-19 10:11:48,409 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-06-19 10:11:48,580 : INFO : built Dictionary(11105 unique tokens: ['15%', '18%', '46%', '一周', '世界']...) from 1005 documents (total 200743 corpus positions)
2020-06-19 10:11:48,581 : INFO : saving Dictionary object under ../data/first_1000_doc.dict, separately None
2020-06-19 10:11:48,587 : INFO : saved ../data/first_1000_doc.dict


Dictionary(11105 unique tokens: ['15%', '18%', '46%', '一周', '世界']...)


In [7]:
corpus = [dictionary.doc2bow(text) for text in texts]
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('15%', 1),
  ('18%', 1),
  ('46%', 1),
  ('一周', 1),
  ('世界', 1),
  ('世界卫生组织', 1),
  ('严峻', 1),
  ('主任', 1),
  ('人数', 1),
  ('传播', 2),
  ('例', 1),
  ('保证', 3),
  ('儿童', 1),
  ('克卢格', 4),
  ('前提', 1),
  ('办事处', 1),
  ('办法', 1),
  ('动员', 1),
  ('区域', 2),
  ('医疗卫生', 4),
  ('协作', 1),
  ('占', 1),
  ('双轨制', 2),
  ('反复', 1),
  ('各国', 1),
  ('同期', 1),
  ('呼吁', 2),
  ('国家', 1),
  ('寻求', 1),
  ('常规', 3),
  ('应', 1),
  ('应对', 1),
  ('弹性', 1),
  ('形势', 1),
  ('快速', 1),
  ('恢复', 1),
  ('抗击', 1),
  ('指出', 1),
  ('控制', 2),
  ('政府', 1),
  ('数', 1),
  ('新冠', 5),
  ('新华社', 1),
  ('日', 1),
  ('日电', 1),
  ('时', 2),
  ('月', 2),
  ('服务', 2),
  ('欧洲', 1),
  ('欧洲各国', 1),
  ('欧洲地区', 2),
  ('死亡', 3),
  ('波', 2),
  ('消失', 1),
  ('特别', 2),
  ('理解', 1),
  ('疫情', 5),
  ('疫苗', 1),
  ('病例', 5),
  ('病毒', 1),
  ('相关', 1),
  ('短时期', 1),
  ('确保', 1),
  ('确诊', 3),
  ('社会', 1),
  ('社区', 2),
  ('第三', 1),
  ('系统', 2),
  ('累计', 4),
  ('视频', 1),
  ('认知', 1),
  ('记者', 1),
  ('说', 2),
  ('达', 2),
  ('运转', 1),
  ('重要性', 1)]]

In [8]:
tf_idf = models.TfidfModel(corpus)


2020-06-19 10:14:03,465 : INFO : collecting document frequencies
2020-06-19 10:14:03,466 : INFO : PROGRESS: processing document #0
2020-06-19 10:14:03,485 : INFO : calculating IDF weights for 1005 documents and 11105 features (109589 matrix non-zeros)


In [10]:
transformed_tfidf = tf_idf[corpus]

In [27]:
%time lda = models.LdaModel(transformed_tfidf, num_topics=8, id2word=dictionary)

2020-06-19 10:40:26,503 : INFO : using symmetric alpha at 0.125
2020-06-19 10:40:26,505 : INFO : using symmetric eta at 0.125
2020-06-19 10:40:26,507 : INFO : using serial LDA version on this node
2020-06-19 10:40:26,520 : INFO : running online (single-pass) LDA training, 8 topics, 1 passes over the supplied corpus of 1005 documents, updating model once every 1005 documents, evaluating perplexity every 1005 documents, iterating 50x with a convergence threshold of 0.001000
2020-06-19 10:40:26,521 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2020-06-19 10:40:27,943 : INFO : -23.143 per-word bound, 9265174.1 perplexity estimate based on a held-out corpus of 1005 documents with 6697 words
2020-06-19 10:40:27,943 : INFO : PROGRESS: pass 0, at document #1005/1005
2020-06-19 10:40:28,393 : INFO : topic #3 (0.125): 0.002*"上年" + 0.002*"同比" + 0.002*"挖贝网" + 0.002*"同期" + 0.002*"报告" + 0.001*"第一季度" + 0.001*"公司" +

CPU times: user 1.88 s, sys: 21.4 ms, total: 1.9 s
Wall time: 1.9 s


In [28]:
lda.show_topics()

[(0,
  '0.002*"同比" + 0.002*"公司" + 0.002*"上年" + 0.001*"减少" + 0.001*"同期" + 0.001*"报告" + 0.001*"本期" + 0.001*"营业" + 0.001*"挖贝网" + 0.001*"增长"'),
 (1,
  '0.002*"同比" + 0.002*"减少" + 0.002*"挖贝网" + 0.002*"公司" + 0.002*"同期" + 0.002*"上年" + 0.001*"报告" + 0.001*"增长" + 0.001*"第一季度" + 0.001*"收入"'),
 (2,
  '0.002*"公司" + 0.002*"同比" + 0.001*"减少" + 0.001*"增长" + 0.001*"收入" + 0.001*"财经" + 0.001*"基金" + 0.001*"归母" + 0.001*"营业" + 0.001*"上年"'),
 (3,
  '0.002*"上年" + 0.002*"同比" + 0.002*"挖贝网" + 0.002*"同期" + 0.002*"报告" + 0.001*"第一季度" + 0.001*"公司" + 0.001*"所致" + 0.001*"减少" + 0.001*"期内"'),
 (4,
  '0.001*"同比" + 0.001*"公司" + 0.001*"疫情" + 0.001*"中国" + 0.001*"下滑" + 0.001*"报告" + 0.001*"增长" + 0.001*"上年" + 0.001*"项目" + 0.001*"美国"'),
 (5,
  '0.002*"公司" + 0.002*"同比" + 0.002*"归母" + 0.002*"净利润" + 0.001*"增长" + 0.001*"年" + 0.001*"下滑" + 0.001*"营收" + 0.001*"产品" + 0.001*"报告"'),
 (6,
  '0.002*"公司" + 0.001*"减少" + 0.001*"同比" + 0.001*"项目" + 0.001*"美国" + 0.001*"年度" + 0.001*"增长" + 0.001*"企业" + 0.001*"全景网" + 0.001*"市场"'),
 (7,
  '0.002*"同比" 

In [16]:
import pyLDAvis
import pyLDAvis.gensim

In [31]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.021125  0.016011       1        1  15.170238
3     -0.018534  0.012266       2        1  14.902268
7      0.005219  0.002563       3        1  14.584406
1     -0.011770  0.007548       4        1  13.088493
6      0.002207 -0.009858       5        1  12.865859
4      0.009580 -0.010280       6        1  11.575086
5     -0.004696 -0.017669       7        1  10.859373
0     -0.003132 -0.000581       8        1   6.954282, topic_info=                                                    Term        Freq  \
10563  andconsiderupgradingtoawebbrowserthatsupportsH...   69.000000   
10562              ToviewthisvideopleaseenableJavaScript   69.000000   
6256                                                  同增   37.000000   
247                                                   上年  216.000000   
1075                                                  承销   33.000000   
...                                                  ...         ...   
481                                                   业务   11.846541   
906                                                   项目   10.864606   
698                                                 上市公司   10.699481   
691                                                  一季度    9.870256   
644                                                   美国    9.819965   

            Total Category  logprob  loglift  
10563   69.000000  Default  30.0000  30.0000  
10562   69.000000  Default  29.0000  29.0000  
6256    37.000000  Default  28.0000  28.0000  
247    216.000000  Default  27.0000  27.0000  
1075    33.000000  Default  26.0000  26.0000  
...           ...      ...      ...      ...  
481    172.408173   Topic8  -7.0719  -0.0120  
906    129.161087   Topic8  -7.1585   0.1903  
698    144.747681   Topic8  -7.1738   0.0610  
691    126.182869   Topic8  -7.2544   0.1176  
644    137.384064   Topic8  -7.2596   0.0274  

[587 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
8538       1  0.169165  0.01
8538       2  0.084583  0.01
8538       3  0.084583  0.01
8538       4  0.422913  0.01
8538       5  0.084583  0.01
...      ...       ...   ...
10536      4  0.054240    鼎汉
10536      5  0.054240    鼎汉
10536      6  0.054240    鼎汉
10536      7  0.054240    鼎汉
10536      8  0.054240    鼎汉

[3104 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 8, 2, 7, 5, 6, 1])

In [45]:
import re
import jieba
from gensim import corpora, models, similarities
from pprint import pprint
from collections import defaultdict
import pyLDAvis
import pyLDAvis.gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def load_stopwords(stopwords_path):
    with open(stopwords_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f]
    
def preprocess_data(corpus_path, stopwords):
    corpus = []
    with open(corpus_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            line = re.sub(r"[0-9\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", line)
            corpus.append(' '.join([word for word in jieba.lcut(line) if word != " " and word != "\t" and word not in stopwords]))
    return corpus

In [46]:
stopwords_path = "../data/stop_words.txt"
documents_path = "../data/documents_股价上涨.txt"
stopwords = load_stopwords(stopwords_path)
documents = preprocess_data(documents_path, stopwords)

In [48]:
documents[0]

'疫情 导致 停工 停产 影响 发酵 新华社 月 日 消息 联合国人口基金 日 公布 分析 报告 称 新冠 疫情 加剧 全球 超过 妇女 避孕药 具 未来 数月 妇女 意外 怀孕 报告 指出 疫情 期间 卫生系统 不堪重负 医疗 设施 关闭 或仅 提供 有限 服务 妇女 担心 感染 病毒 选择 暂时 放弃 体检 供应链 中断 导致 避孕药 具 短缺 妇女 长期 被困 家中 避孕药 具 天然 胶乳 安全套 生产 原料 橡胶 王国 马来西亚 新冠 疫情 安全套 产业链 连锁 影响 月 日 中午 马来西亚 新增 确诊 病例 例 累计 确诊 病例 例 病死率 累计 治愈 例 接近 确诊 病例 成 时间 月 日 马来西亚 卫生部 颁布 第四阶段 管制 令及 实施方案 政府 放宽 民众 一名 家人 陪伴 外出 前往 住所 十公里 购买 食物 生活必需品 药物 营养品 马来西亚 政府 遏制 新冠 肺炎 疫情 颁布 第四阶段 管制 令 有效期 是从 月 日至 月 日 疫情 影响 位于 马来西亚 全球 避孕套 生产商 康乐 公司 此前 被迫 停止 生产 恢复 月底 警告 市场供应 减少 亿个 安全套 缺口 康乐 公司 库存 足以 应付 月 需求 供应 端 承压 需求 端则 增加 电商 平台 数据 显示 疫情 期间 安全套 销量 大幅 增长 人福 医药 股价 一个月 大涨 月初 康乐 马来西亚 政府 豁免 重启 厂房 只能 成 劳动力 意味着 马来西亚 安全套 产量 短时间 内会 大幅 下降 启动 工厂 时间 难 产能 减半 情况 跟上 需求 康乐 公司 首席 执行官 预测 避孕套 全球 短缺 非洲地区 短缺 长 达 数月 久 疫情 影响 之下 行业 参与者 纷纷 采取措施 应对 乐福思 集团 全球 副总裁 杰士邦 总经理 张文耀 接受 记者 采访 时 全球 拥有 安全套 生产 基地 很强 自主 调节 能力 中国 地区 杰士邦 产品 泰国 日本 印度 工厂 供应 中国 疫情 爆发 第一 时间 乐福思 集团 杰士邦 公司 高度 关注 全球 着眼 做 应对 预案 加大 原料 成品 库存 储备 供应 短缺 疫情 影响 公司 早 一月份 调整 供应链 策略 市场策略 增加 库存 应对 变化 市场需求 泰国 日本 工厂 供应 疫情 影响 印度 工厂 受 疫情 影响 停工 中国 市场 品规 短期 

In [49]:
texts = [[word for word in document.split()] for document in documents]

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 2] for text in texts]

dictionary = corpora.Dictionary(texts)
dictionary.save('../data/documents_估价上涨_doc.dict')
print(dictionary)

2020-06-19 13:43:54,972 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-06-19 13:43:55,624 : INFO : adding document #10000 to Dictionary(13907 unique tokens: ['一个月', '一名', '一季度', '一家', '一月']...)
2020-06-19 13:43:55,670 : INFO : built Dictionary(13982 unique tokens: ['一个月', '一名', '一季度', '一家', '一月']...) from 10763 documents (total 779833 corpus positions)
2020-06-19 13:43:55,670 : INFO : saving Dictionary object under ../data/documents_估价上涨_doc.dict, separately None
2020-06-19 13:43:55,679 : INFO : saved ../data/documents_估价上涨_doc.dict


Dictionary(13982 unique tokens: ['一个月', '一名', '一季度', '一家', '一月']...)


In [50]:
corpus = [dictionary.doc2bow(text) for text in texts]
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]]

tf_idf = models.TfidfModel(corpus)
transformed_tfidf = tf_idf[corpus]
%time lda = models.LdaModel(transformed_tfidf, num_topics=8, id2word=dictionary)

2020-06-19 13:44:26,244 : INFO : collecting document frequencies
2020-06-19 13:44:26,245 : INFO : PROGRESS: processing document #0
2020-06-19 13:44:26,330 : INFO : PROGRESS: processing document #10000
2020-06-19 13:44:26,336 : INFO : calculating IDF weights for 10763 documents and 13982 features (526850 matrix non-zeros)
2020-06-19 13:44:26,399 : INFO : using symmetric alpha at 0.125
2020-06-19 13:44:26,400 : INFO : using symmetric eta at 0.125
2020-06-19 13:44:26,402 : INFO : using serial LDA version on this node
2020-06-19 13:44:26,413 : INFO : running online (single-pass) LDA training, 8 topics, 1 passes over the supplied corpus of 10763 documents, updating model once every 2000 documents, evaluating perplexity every 10763 documents, iterating 50x with a convergence threshold of 0.001000
2020-06-19 13:44:26,413 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2020-06-19 13:44:26,797 : INFO : PROGRESS

CPU times: user 5.16 s, sys: 52.4 ms, total: 5.21 s
Wall time: 5.23 s


In [51]:
lda.show_topics()

[(0,
  '0.010*"股份" + 0.009*"继峰" + 0.008*"华鑫" + 0.007*"长峰" + 0.006*"米业" + 0.006*"金健" + 0.006*"航天" + 0.006*"京粮" + 0.005*"路" + 0.005*"达"'),
 (1,
  '0.011*"药业" + 0.009*"佳" + 0.008*"涨停" + 0.006*"世纪" + 0.006*"达" + 0.005*"点" + 0.005*"分钟" + 0.005*"快照" + 0.005*"分盘口" + 0.005*"盘中"'),
 (2,
  '0.017*"东方" + 0.009*"文化" + 0.008*"股份" + 0.007*"达" + 0.007*"锐电" + 0.006*"分钟" + 0.006*"分盘口" + 0.006*"快照" + 0.006*"点" + 0.005*"新能"'),
 (3,
  '0.009*"达" + 0.009*"集团" + 0.008*"分钟" + 0.007*"分盘口" + 0.007*"快照" + 0.007*"点" + 0.007*"快速" + 0.007*"珠宝" + 0.007*"盘中" + 0.007*"超过"'),
 (4,
  '0.012*"证券" + 0.012*"席位" + 0.009*"营业部" + 0.008*"当日" + 0.007*"偏离" + 0.006*"实力" + 0.006*"买入" + 0.006*"显示" + 0.006*"卖出" + 0.005*"值达"'),
 (5,
  '0.009*"流出" + 0.007*"个股" + 0.007*"次" + 0.007*"主力" + 0.006*"净流入" + 0.006*"金融界" + 0.006*"截止" + 0.006*"网站" + 0.005*"年" + 0.005*"公司"'),
 (6,
  '0.028*"股份" + 0.019*"科技" + 0.016*"达" + 0.016*"分钟" + 0.014*"快速" + 0.014*"超过" + 0.014*"点" + 0.014*"快照" + 0.014*"分盘口" + 0.013*"盘中"'),
 (7,
  '0.006*"涨" + 0.004*"黄金" + 

In [52]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.165134 -0.164286       1        1  30.101582
7      0.114610  0.115490       2        1  25.588352
6     -0.160556 -0.024856       3        1  21.202021
4      0.073360  0.113122       4        1  13.392216
1     -0.017451 -0.015756       5        1   5.445655
0     -0.020927 -0.013863       6        1   1.636660
3     -0.100393 -0.010909       7        1   1.412168
2     -0.053777  0.001058       8        1   1.221341, topic_info=     Term         Freq        Total Category  logprob  loglift
663    股份  6775.000000  6775.000000  Default  30.0000  30.0000
383     达  4335.000000  4335.000000  Default  29.0000  29.0000
495    分钟  4066.000000  4066.000000  Default  28.0000  28.0000
1443  分盘口  3542.000000  3542.000000  Default  27.0000  27.0000
1445   快照  3542.000000  3542.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
212     报    43.143852  3033.435303   Topic8  -5.3971   0.1523
54     信息    42.112858  2948.655762   Topic8  -5.4213   0.1565
11     上涨    41.915760  3852.051514   Topic8  -5.4260  -0.1155
878   换手率    41.174862  2814.585205   Topic8  -5.4438   0.1805
1444   异动    40.491367  2728.357666   Topic8  -5.4605   0.1949

[547 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.919609  一个月
0         2  0.036253  一个月
0         3  0.010876  一个月
0         4  0.012084  一个月
0         5  0.009667  一个月
...     ...       ...  ...
1938      4  0.404225  龙虎榜
1938      5  0.005192  龙虎榜
1938      6  0.012609  龙虎榜
1938      7  0.000742  龙虎榜
1938      8  0.001483  龙虎榜

[2725 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 7, 5, 2, 1, 4, 3])